In [1]:
# ============================================================
# 0) Imports + config
# ============================================================
import os
import zipfile
import tempfile
from pathlib import Path

import numpy as np
import xarray as xr
import rioxarray  # adds .rio accessor
import geopandas as gpd
from shapely.geometry import box

from rasterio.enums import Resampling
from rasterio.transform import from_origin

import fsspec
import gcsfs

# Dask / distributed
from dask_gateway import Gateway
from dask.distributed import Client

# ------------------------------------------------------------
# LEAP scratch bucket
# ------------------------------------------------------------
SCRATCH_BUCKET = os.environ.get("SCRATCH_BUCKET", "gs://leap-scratch/renriviera")
OUT_PREFIX = f"{SCRATCH_BUCKET.rstrip('/')}/sfincs_soundview_preproc"
print("Writing outputs to:", OUT_PREFIX)

# Target grid settings (UTM zone 18N is standard for NYC)
TARGET_CRS = "EPSG:26918"  # NAD83 / UTM zone 18N (meters)
TARGET_RES = 25  # meters (try 10, 25, or 50)

# ============================================================
# 1) Spin up a Dask cluster (Gateway) - do this BEFORE heavy work
# ============================================================
gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=2, maximum=10)  # adjust as needed
client = cluster.get_client()
client


Writing outputs to: gs://leap-scratch/renriviera/sfincs_soundview_preproc


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/prod.8cb1ef7cad0e44d99fe059ad7d0818eb/status,


2026-01-16 21:48:55,062 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [2]:
# ============================================================
# 2) Define ROI (Bronx or Soundview bbox)
# ============================================================

# Quick bbox for Soundview-ish area (you can refine):
# (lon_min, lat_min, lon_max, lat_max)
soundview_bbox_wgs84 = (-73.882, 40.807, -73.842, 40.836)

roi_geom_wgs84 = box(*soundview_bbox_wgs84)
roi = gpd.GeoDataFrame({"name": ["soundview_bbox"]}, geometry=[roi_geom_wgs84], crs="EPSG:4326")
roi_utm = roi.to_crs(TARGET_CRS)
roi_utm


,name,geometry
0,soundview_bbox,"POLYGON ((597674.087 4517977.754, 597631.547 4..."


In [3]:
# ============================================================
# 3) Download helper functions
# ============================================================
import requests


def download_file(url: str, dst_path: Path, chunk_size: int = 1024 * 1024):
    dst_path.parent.mkdir(parents=True, exist_ok=True)
    print("Downloading:", url)
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(dst_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=chunk_size):
                if chunk:
                    f.write(chunk)
    return dst_path

def unzip_to_dir(zip_path: Path, out_dir: Path):
    out_dir.mkdir(parents=True, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(out_dir)
    return out_dir

# NYC datasets (large!)
NYC_LANDCOVER_ZIP = "https://data.cityofnewyork.us/download/he6d-2qns/application/zip"
NYC_DEM_INT_ZIP   = "https://sa-static-customer-assets-us-east-1-fedramp-prod.s3.amazonaws.com/data.cityofnewyork.us/NYC_DEM_1ft_Int.zip"


In [4]:
# ============================================================
# 4) Raster preprocessing utilities
# ============================================================

def open_raster_lazy(path: Path, chunks="auto"):
    # rioxarray.open_rasterio creates a DataArray with dims: band, y, x
    da = rioxarray.open_rasterio(path, chunks=chunks, masked=True)
    # Drop band dim if single-band
    if "band" in da.dims and da.sizes["band"] == 1:
        da = da.squeeze("band", drop=True)
    return da

def clip_to_roi(da: xr.DataArray, roi_gdf_utm: gpd.GeoDataFrame):
    # Clip expects shapes in same CRS
    if da.rio.crs is None:
        raise ValueError("Raster has no CRS. You must assign one before clipping.")
    roi_same = roi_gdf_utm.to_crs(da.rio.crs)
    return da.rio.clip(roi_same.geometry, all_touched=True, drop=True)

def make_template_grid_from_bbox(roi_utm, res_m: float, crs: str):
    """
    Create a canonical template grid in a projected CRS (meters).
    Ensures:
      - fixed resolution (dx=dy=res_m)
      - snapped extent (alignment consistency)
      - valid affine transform
    """
    minx, miny, maxx, maxy = roi_utm.total_bounds

    # snap bounds to resolution for consistent alignment
    def snap_floor(v): return np.floor(v / res_m) * res_m
    def snap_ceil(v):  return np.ceil(v / res_m) * res_m

    minx_s, miny_s = snap_floor(minx), snap_floor(miny)
    maxx_s, maxy_s = snap_ceil(maxx), snap_ceil(maxy)

    width  = int((maxx_s - minx_s) / res_m)
    height = int((maxy_s - miny_s) / res_m)

    # Pixel centers
    x = minx_s + (np.arange(width) + 0.5) * res_m
    y = maxy_s - (np.arange(height) + 0.5) * res_m  # top-down (north-up)

    template = xr.DataArray(
        np.full((height, width), np.nan, dtype=np.float32),
        coords={"y": y, "x": x},
        dims=("y", "x"),
        name="template"
    )

    # Affine transform: upper-left corner (minx_s, maxy_s), pixel size res_m
    transform = from_origin(minx_s, maxy_s, res_m, res_m)

    template = template.rio.write_crs(crs)
    template = template.rio.write_transform(transform)

    return template

def reproject_match(da: xr.DataArray, template: xr.DataArray, resampling: Resampling):
    return da.rio.reproject_match(template, resampling=resampling)

def write_cog_local(da: xr.DataArray, out_path: Path, nodata=None):
    out_path.parent.mkdir(parents=True, exist_ok=True)
    if nodata is not None:
        da = da.rio.write_nodata(nodata)
    da.rio.to_raster(
        out_path,
        driver="COG",
        compress="deflate",
        BIGTIFF="IF_SAFER",
        tiled=True
    )
    return out_path

def gcs_upload(local_path: Path, gcs_url: str):
    fs = gcsfs.GCSFileSystem()
    with open(local_path, "rb") as fsrc:
        with fs.open(gcs_url.replace("gs://", ""), "wb") as fdst:
            fdst.write(fsrc.read())
    return gcs_url


In [5]:
# ============================================================
# 5) Download + preprocess DEM and Land Cover  (DROP-IN REPLACEMENT)
#     DEM: NYC 1-ft DEM ZIP  -> clip (local) -> UTM -> 25m (bilinear)
#     Landcover: ESA WorldCover 10m COG (HTTPS) -> clip -> UTM -> 25m (nearest)
#     Outputs written to: gs://leap-scratch/... (NOT /home/Romain)
#
# FIXES INCLUDED:
#   - DEM: download/unzip in /tmp, then clip + .load() BEFORE reproject
#          so Dask workers never touch /tmp paths.
#   - WorldCover: DO NOT use s3:// (GDAL S3 auth issues on LEAP).
#          Use HTTPS public URL instead (no credentials).
# ============================================================

import os
import tempfile
from pathlib import Path

import numpy as np
import rioxarray
import rasterio
from rasterio.enums import Resampling
from rasterio.transform import from_origin

# ------------------------------------------------------------
# Ensure template exists (canonical UTM grid)
# ------------------------------------------------------------
def make_template_grid_from_bbox(roi_utm, res_m: float, crs: str):
    minx, miny, maxx, maxy = roi_utm.total_bounds

    def snap_floor(v): return np.floor(v / res_m) * res_m
    def snap_ceil(v):  return np.ceil(v / res_m) * res_m

    minx_s, miny_s = snap_floor(minx), snap_floor(miny)
    maxx_s, maxy_s = snap_ceil(maxx), snap_ceil(maxy)

    width  = int((maxx_s - minx_s) / res_m)
    height = int((maxy_s - miny_s) / res_m)

    # Pixel centers
    x = minx_s + (np.arange(width) + 0.5) * res_m
    y = maxy_s - (np.arange(height) + 0.5) * res_m  # north-up

    import xarray as xr
    template = xr.DataArray(
        np.full((height, width), np.nan, dtype=np.float32),
        coords={"y": y, "x": x},
        dims=("y", "x"),
        name="template",
    )

    transform = from_origin(minx_s, maxy_s, res_m, res_m)
    template = template.rio.write_crs(crs)
    template = template.rio.write_transform(transform)
    return template


if "template" not in globals():
    print("template not found - rebuilding template grid now...")
    template = make_template_grid_from_bbox(roi_utm, TARGET_RES, TARGET_CRS)

print("Template CRS:", template.rio.crs, "res:", template.rio.resolution(), "shape:", template.shape)

# ------------------------------------------------------------
# Output locations on scratch
# ------------------------------------------------------------
rasters_prefix = f"{OUT_PREFIX}/rasters"
print("Scratch prefix:", rasters_prefix)

# ------------------------------------------------------------
# Use /tmp (ephemeral) for downloads/extraction, NOT /home
# ------------------------------------------------------------
workdir = Path(tempfile.mkdtemp(prefix="sfincs_preproc_"))
raw_dir = workdir / "raw"
proc_dir = workdir / "processed"
raw_dir.mkdir(exist_ok=True, parents=True)
proc_dir.mkdir(exist_ok=True, parents=True)

print("Temp workdir:", workdir)

# ============================================================
# (A) DEM: NYC 1-ft DEM ZIP
# ============================================================
NYC_DEM_INT_ZIP = "https://sa-static-customer-assets-us-east-1-fedramp-prod.s3.amazonaws.com/data.cityofnewyork.us/NYC_DEM_1ft_Int.zip"

dem_zip = download_file(NYC_DEM_INT_ZIP, raw_dir / "nyc_dem_int.zip")
dem_unzipped = unzip_to_dir(dem_zip, raw_dir / "dem_int")

dem_tifs = list(dem_unzipped.rglob("*.tif")) + list(dem_unzipped.rglob("*.tiff"))
if len(dem_tifs) == 0:
    raise RuntimeError("No DEM GeoTIFF found after unzip.")

dem_path = dem_tifs[0]
print("DEM GeoTIFF:", dem_path)

# IMPORTANT: DEM is local on /tmp -> open without Dask chunks
dem_raw = rioxarray.open_rasterio(dem_path, masked=True).squeeze()

print("DEM CRS:", dem_raw.rio.crs)
print("ROI UTM CRS:", roi_utm.crs)
print("DEM bounds:", dem_raw.rio.bounds())
print("ROI UTM bounds:", roi_utm.total_bounds)

# Clip in DEM CRS (EPSG:2263), not in UTM
roi_in_dem_crs = roi_utm.to_crs(dem_raw.rio.crs)
minx, miny, maxx, maxy = roi_in_dem_crs.total_bounds

dem_clip = dem_raw.rio.clip_box(minx=minx, miny=miny, maxx=maxx, maxy=maxy)

# CRITICAL: load into notebook memory before reprojection
dem_clip = dem_clip.load()

# Reproject to template (continuous -> bilinear)
dem_utm = dem_clip.rio.reproject_match(template, resampling=Resampling.bilinear)

# Make sure we can write safely (float + explicit nodata)
DEM_NODATA = -9999.0
dem_utm = dem_utm.astype("float32")
dem_utm = dem_utm.rio.write_nodata(DEM_NODATA, encoded=True)
dem_utm = dem_utm.fillna(DEM_NODATA)

# Write DEM COG locally -> upload to scratch
dem_out_local = proc_dir / f"dep_dem_utm{TARGET_RES}m.tif"
dem_utm.rio.to_raster(
    dem_out_local,
    driver="COG",
    dtype="float32",
    compress="deflate",
    BIGTIFF="IF_SAFER",
    tiled=True,
    nodata=DEM_NODATA,
)

dem_out_gcs = f"{rasters_prefix}/dep_dem_utm{TARGET_RES}m.tif"
gcs_upload(dem_out_local, dem_out_gcs)
print("✅ Wrote DEM to:", dem_out_gcs)

# ============================================================
# (B) Land Cover: ESA WorldCover 10m COG via HTTPS (NO AWS AUTH)
# ============================================================
# Use HTTPS endpoint to avoid GDAL S3 auth issues in this environment.
WC_BASE = "https://esa-worldcover.s3.eu-central-1.amazonaws.com/v200/2021/map"
candidate_tiles = ["N39W075", "N42W075", "N39W078", "N42W078"]

def try_open_worldcover_https(tile):
    url = f"{WC_BASE}/ESA_WorldCover_10m_2021_v200_{tile}_Map.tif"
    # Rasterio/GDAL speed tweak for remote COGs
    with rasterio.Env(GDAL_DISABLE_READDIR_ON_OPEN="YES"):
        da = rioxarray.open_rasterio(
            url,
            masked=True,
            # keep small chunks so compute happens locally without needing a Dask cluster
            chunks={"x": 1024, "y": 1024},
            lock=False,
        ).squeeze()
        _ = da.rio.bounds()
    return da, url

wc_raw, wc_url = None, None
for tile in candidate_tiles:
    try:
        wc_raw, wc_url = try_open_worldcover_https(tile)
        print("✅ Opened WorldCover tile (HTTPS):", wc_url)
        break
    except Exception as e:
        print(f"WorldCover HTTPS tile failed {tile}: {type(e).__name__}")

if wc_raw is None:
    raise RuntimeError("Could not open any WorldCover HTTPS tile candidates. Expand candidate list.")

# Clip WorldCover in EPSG:4326 bounds (WorldCover is lat/lon)
roi_wgs84 = roi_utm.to_crs("EPSG:4326")
minlon, minlat, maxlon, maxlat = roi_wgs84.total_bounds

wc_clip = wc_raw.rio.clip_box(minx=minlon, miny=minlat, maxx=maxlon, maxy=maxlat)

# Reproject to template (categorical -> nearest)
lc_utm = wc_clip.rio.reproject_match(template, resampling=Resampling.nearest)

# IMPORTANT: compute locally (NO distributed client required)
lc_utm = lc_utm.compute()

# Write landcover COG locally -> upload to scratch
lc_out_local = proc_dir / f"landcover_worldcover_utm{TARGET_RES}m.tif"
lc_utm.rio.to_raster(
    lc_out_local,
    driver="COG",
    dtype="uint8",
    compress="deflate",
    BIGTIFF="IF_SAFER",
    tiled=True,
    nodata=0,
)

lc_out_gcs = f"{rasters_prefix}/landcover_worldcover_utm{TARGET_RES}m.tif"
gcs_upload(lc_out_local, lc_out_gcs)
print("✅ Wrote Landcover (WorldCover) to:", lc_out_gcs)

# ------------------------------------------------------------
# Quick sanity check: confirm alignment
# ------------------------------------------------------------
print("\n--- Alignment sanity ---")
print("DEM CRS:", dem_utm.rio.crs, "res:", dem_utm.rio.resolution(), "shape:", dem_utm.shape, "dtype:", dem_utm.dtype)
print("LC  CRS:", lc_utm.rio.crs,  "res:", lc_utm.rio.resolution(),  "shape:", lc_utm.shape,  "dtype:", lc_utm.dtype)




template not found - rebuilding template grid now...
Template CRS: EPSG:26918 res: (25.0, -25.0) shape: (131, 137)
Scratch prefix: gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters
Temp workdir: /tmp/sfincs_preproc_c72wj__9
Downloading: https://sa-static-customer-assets-us-east-1-fedramp-prod.s3.amazonaws.com/data.cityofnewyork.us/NYC_DEM_1ft_Int.zip
DEM GeoTIFF: /tmp/sfincs_preproc_c72wj__9/raw/dem_int/DEM_LiDAR_1ft_2010_Improved_NYC_int.tif
DEM CRS: EPSG:2263
ROI UTM CRS: EPSG:26918
DEM bounds: (910719.3, 119060.67499999999, 1068819.3, 275160.675)
ROI UTM bounds: [ 594259.050855   4517933.95713113  597674.08711018 4521197.02053401]
✅ Wrote DEM to: gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/dep_dem_utm25m.tif
✅ Opened WorldCover tile (HTTPS): https://esa-worldcover.s3.eu-central-1.amazonaws.com/v200/2021/map/ESA_WorldCover_10m_2021_v200_N39W075_Map.tif
✅ Wrote Landcover (WorldCover) to: gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/landco

In [6]:
# ============================================================
# 6) Derived features for SFINCS (DROP-IN REPLACEMENT)
#     - Manning roughness (from ESA WorldCover classes)
#     - Impervious fraction (heuristic from landcover)
#     - Curve Number CN (simple heuristic from landcover)
#
# KEY FIX:
#   - Force DEM + LC to .load() (in-memory) before any computation
#   - Ensure derived rasters are NOT Dask-backed before .to_raster()
# ============================================================

import tempfile
from pathlib import Path

import numpy as np
import rioxarray

# ------------------------------------------------------------
# Inputs from scratch (written by Cell 5)
# ------------------------------------------------------------
dem_gcs = f"{OUT_PREFIX}/rasters/dep_dem_utm{TARGET_RES}m.tif"
lc_gcs  = f"{OUT_PREFIX}/rasters/landcover_worldcover_utm{TARGET_RES}m.tif"

print("Reading DEM:", dem_gcs)
print("Reading LC :", lc_gcs)

dem = rioxarray.open_rasterio(dem_gcs, masked=True).squeeze()
lc  = rioxarray.open_rasterio(lc_gcs,  masked=True).squeeze()

print("DEM CRS:", dem.rio.crs, "shape:", dem.shape, "chunks:", getattr(dem.data, "chunks", None))
print("LC  CRS:", lc.rio.crs,  "shape:", lc.shape,  "chunks:", getattr(lc.data,  "chunks", None))

# ------------------------------------------------------------
# CRITICAL: Materialize both into memory (Soundview is small)
# This prevents rioxarray from trying to re-open things from /tmp during writes
# ------------------------------------------------------------
dem = dem.load()
lc  = lc.load()

# Standardize dtypes
dem = dem.astype("float32")
lc  = lc.astype("int16")

# ------------------------------------------------------------
# WorldCover 10m class codes (v200):
# 10 Tree cover
# 20 Shrubland
# 30 Grassland
# 40 Cropland
# 50 Built-up
# 60 Bare / sparse vegetation
# 70 Snow and ice
# 80 Permanent water bodies
# 90 Herbaceous wetland
# 95 Mangroves
# 100 Moss and lichen
# ------------------------------------------------------------

# ------------------------------------------------------------
# (A) Manning roughness lookup (reasonable first-pass values)
# Units: s / m^(1/3)
# ------------------------------------------------------------
manning_lut = {
    10: 0.10,   # Tree cover
    20: 0.07,   # Shrubland
    30: 0.05,   # Grassland
    40: 0.04,   # Cropland
    50: 0.02,   # Built-up (smooth-ish, but varies a lot)
    60: 0.03,   # Bare/sparse
    70: 0.02,   # Snow/ice
    80: 0.025,  # Water
    90: 0.08,   # Wetland
    95: 0.10,   # Mangroves
    100: 0.10,  # Moss/lichen
}

# vectorized mapping: default fallback = 0.05
manning = np.full(lc.shape, 0.05, dtype=np.float32)
for k, v in manning_lut.items():
    manning = np.where(lc.values == k, np.float32(v), manning)

manning = lc.copy(data=manning).astype("float32")
manning.name = "manning"

# ------------------------------------------------------------
# (B) Impervious fraction (heuristic)
# Built-up -> high impervious; other classes lower
# ------------------------------------------------------------
imperv_lut = {
    50: 0.85,  # Built-up
    40: 0.15,  # Cropland
    60: 0.05,  # Bare
    30: 0.05,  # Grass
    20: 0.05,  # Shrub
    10: 0.02,  # Trees
    90: 0.02,  # Wetland
    80: 0.00,  # Water
    70: 0.00,
    95: 0.02,
    100: 0.02,
}

impervious = np.full(lc.shape, 0.05, dtype=np.float32)  # default small
for k, v in imperv_lut.items():
    impervious = np.where(lc.values == k, np.float32(v), impervious)

impervious = lc.copy(data=impervious).astype("float32")
impervious.name = "impervious_frac"

# ------------------------------------------------------------
# (C) Curve Number CN (simple heuristic, AMC II)
# NOTE: This is a "good enough" baseline without soils/HSG.
# For real work, replace with SSURGO HSG + NLCD-based CN tables.
# ------------------------------------------------------------
cn_lut = {
    50: 92,  # Built-up (impervious dominated)
    40: 78,  # Cropland
    30: 74,  # Grassland
    20: 70,  # Shrubland
    10: 60,  # Tree cover
    60: 80,  # Bare/sparse
    90: 85,  # Wetland (often saturated)
    80: 98,  # Water
    70: 90,
    95: 85,
    100: 70,
}

cn = np.full(lc.shape, 75, dtype=np.float32)  # default
for k, v in cn_lut.items():
    cn = np.where(lc.values == k, np.float32(v), cn)

cn = lc.copy(data=cn).astype("float32")
cn.name = "curve_number"

# ------------------------------------------------------------
# Attach consistent nodata + fill NaNs before writing
# ------------------------------------------------------------
OUT_NODATA = -9999.0
for da in [manning, impervious, cn]:
    da = da.astype("float32")
    da = da.rio.write_nodata(OUT_NODATA, encoded=True)
    da = da.fillna(OUT_NODATA)

# (re-assign after loop because xarray objects are immutable-ish in loop variable)
manning    = manning.astype("float32").rio.write_nodata(OUT_NODATA, encoded=True).fillna(OUT_NODATA)
impervious = impervious.astype("float32").rio.write_nodata(OUT_NODATA, encoded=True).fillna(OUT_NODATA)
cn         = cn.astype("float32").rio.write_nodata(OUT_NODATA, encoded=True).fillna(OUT_NODATA)

# ------------------------------------------------------------
# Write to /tmp locally, then upload to scratch
# ------------------------------------------------------------
out_local_dir = Path(tempfile.mkdtemp(prefix="sfincs_features_"))
print("Local output dir:", out_local_dir)

manning_local    = out_local_dir / f"manning_n_utm{TARGET_RES}m.tif"
impervious_local = out_local_dir / f"impervious_frac_utm{TARGET_RES}m.tif"
cn_local         = out_local_dir / f"curve_number_cn_utm{TARGET_RES}m.tif"

manning.rio.to_raster(
    manning_local,
    driver="COG",
    dtype="float32",
    compress="deflate",
    BIGTIFF="IF_SAFER",
    tiled=True,
    nodata=OUT_NODATA,
)
impervious.rio.to_raster(
    impervious_local,
    driver="COG",
    dtype="float32",
    compress="deflate",
    BIGTIFF="IF_SAFER",
    tiled=True,
    nodata=OUT_NODATA,
)
cn.rio.to_raster(
    cn_local,
    driver="COG",
    dtype="float32",
    compress="deflate",
    BIGTIFF="IF_SAFER",
    tiled=True,
    nodata=OUT_NODATA,
)

derived_prefix = f"{OUT_PREFIX}/rasters/derived"
manning_gcs    = f"{derived_prefix}/manning_n_utm{TARGET_RES}m.tif"
imperv_gcs     = f"{derived_prefix}/impervious_frac_utm{TARGET_RES}m.tif"
cn_gcs         = f"{derived_prefix}/curve_number_cn_utm{TARGET_RES}m.tif"

gcs_upload(manning_local, manning_gcs)
gcs_upload(impervious_local, imperv_gcs)
gcs_upload(cn_local, cn_gcs)

print("✅ Wrote derived rasters to scratch:")
print(" -", manning_gcs)
print(" -", imperv_gcs)
print(" -", cn_gcs)


Reading DEM: gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/dep_dem_utm25m.tif
Reading LC : gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/landcover_worldcover_utm25m.tif
DEM CRS: EPSG:26918 shape: (131, 137) chunks: None
LC  CRS: EPSG:26918 shape: (131, 137) chunks: None
Local output dir: /tmp/sfincs_features_rg5016bh


/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:250: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


✅ Wrote derived rasters to scratch:
 - gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/derived/manning_n_utm25m.tif
 - gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/derived/impervious_frac_utm25m.tif
 - gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/derived/curve_number_cn_utm25m.tif


In [7]:
import rioxarray

m = rioxarray.open_rasterio(f"{OUT_PREFIX}/rasters/derived/manning_n_utm{TARGET_RES}m.tif").squeeze()
imp = rioxarray.open_rasterio(f"{OUT_PREFIX}/rasters/derived/impervious_frac_utm{TARGET_RES}m.tif").squeeze()
cn = rioxarray.open_rasterio(f"{OUT_PREFIX}/rasters/derived/curve_number_cn_utm{TARGET_RES}m.tif").squeeze()

print("Manning min/max:", float(m.min()), float(m.max()))
print("Imperv min/max :", float(imp.min()), float(imp.max()))
print("CN min/max     :", float(cn.min()), float(cn.max()))


Manning min/max: 0.019999999552965164 0.10000000149011612
Imperv min/max : 0.0 0.8500000238418579
CN min/max     : 60.0 98.0


In [8]:
# ============================================================
# FINAL DEM + FINAL DERIVED (robust):
# 1) Convert DEM feet -> meters + enforce nodata (GTiff writer)
# 2) Repair derived rasters nodata (manning/impervious/CN) if missing
# Writes to: gs://.../rasters/final/
# ============================================================

import tempfile
from pathlib import Path

import numpy as np
import rioxarray
import rasterio

DEM_FEET_TO_M = 0.3048
DEM_NODATA = -9999.0

clean_prefix = f"{OUT_PREFIX}/rasters/clean"
final_prefix = f"{OUT_PREFIX}/rasters/final"

# ------------------------------------------------------------
# Local workdir (ephemeral)
# ------------------------------------------------------------
local_dir = Path(tempfile.mkdtemp(prefix="sfincs_finalize_"))
print("Local workdir:", local_dir)

# ============================================================
# (A) FINAL DEM: feet -> meters + nodata enforced
# ============================================================
dem_in = f"{clean_prefix}/dep_dem_utm{TARGET_RES}m.tif"
dem_out_gcs = f"{final_prefix}/dep_dem_utm{TARGET_RES}m_meters.tif"
dem_out_local = local_dir / f"dep_dem_utm{TARGET_RES}m_meters.tif"

print("\n--- FINAL DEM (feet -> meters) ---")
print("Reading CLEAN DEM:", dem_in)

dem = rioxarray.open_rasterio(dem_in, masked=False).squeeze().load().astype("float32")
arr = np.asarray(dem.values)

# Treat -9999 as nodata even if metadata missing
valid = np.isfinite(arr) & (arr != DEM_NODATA)

arr_m = arr.copy()
arr_m[valid] = arr_m[valid] * DEM_FEET_TO_M
arr_m[~valid] = DEM_NODATA

transform = dem.rio.transform(recalc=True)
crs = dem.rio.crs
height, width = dem.shape

profile = {
    "driver": "GTiff",
    "dtype": "float32",
    "count": 1,
    "height": height,
    "width": width,
    "crs": crs,
    "transform": transform,
    "nodata": float(DEM_NODATA),
    "tiled": True,
    "blockxsize": 512,
    "blockysize": 512,
    "compress": "deflate",
    "bigtiff": "IF_SAFER",
}

print("Writing FINAL DEM locally (GTiff):", dem_out_local)
with rasterio.open(dem_out_local, "w", **profile) as dst:
    dst.write(arr_m.astype("float32"), 1)

with rasterio.open(dem_out_local) as ds:
    print("✅ FINAL DEM local nodata:", ds.nodata)
    if ds.nodata is None or not np.isclose(ds.nodata, DEM_NODATA):
        raise RuntimeError(f"FINAL DEM nodata write failed: got {ds.nodata}, expected {DEM_NODATA}")

gcs_upload(dem_out_local, dem_out_gcs)
print("✅ FINAL DEM (meters) written to:", dem_out_gcs)

# ============================================================
# (B) FINAL DERIVED: repair nodata metadata if missing
# ============================================================

print("\n--- FINAL DERIVED (repair nodata if missing) ---")

derived_clean = {
    "manning":      f"{clean_prefix}/manning_n_utm{TARGET_RES}m.tif",
    "impervious":   f"{clean_prefix}/impervious_frac_utm{TARGET_RES}m.tif",
    "curve_number": f"{clean_prefix}/curve_number_cn_utm{TARGET_RES}m.tif",
}

def _rewrite_with_nodata_if_needed(name: str, src_gcs: str, dst_gcs: str, nodata_value: float):
    """
    Loads a raster, and if nodata is missing, rewrites it as GTiff with nodata enforced.
    Always uploads into final/ (idempotent and safe).
    """
    print(f"\n[{name}] Source:", src_gcs)

    da = rioxarray.open_rasterio(src_gcs, masked=False).squeeze().load()
    src_nodata = da.rio.nodata
    print(f"[{name}] Current nodata:", src_nodata)

    arr = np.asarray(da.values).astype("float32")

    # Always ensure NaNs are removed (important for HydroMT / SFINCS)
    arr = np.where(np.isfinite(arr), arr, nodata_value).astype("float32")

    # If nodata missing, enforce -9999
    # If nodata exists, we still normalize NaNs and enforce output nodata=-9999
    transform = da.rio.transform(recalc=True)
    crs = da.rio.crs
    height, width = arr.shape

    out_local = local_dir / Path(dst_gcs).name

    prof = {
        "driver": "GTiff",
        "dtype": "float32",
        "count": 1,
        "height": height,
        "width": width,
        "crs": crs,
        "transform": transform,
        "nodata": float(nodata_value),
        "tiled": True,
        "blockxsize": 512,
        "blockysize": 512,
        "compress": "deflate",
        "bigtiff": "IF_SAFER",
    }

    print(f"[{name}] Writing fixed raster locally:", out_local)
    with rasterio.open(out_local, "w", **prof) as dst:
        dst.write(arr, 1)

    with rasterio.open(out_local) as ds:
        if ds.nodata is None or not np.isclose(ds.nodata, nodata_value):
            raise RuntimeError(f"[{name}] nodata write failed: got {ds.nodata}, expected {nodata_value}")
        print(f"[{name}] ✅ fixed local nodata:", ds.nodata)

    gcs_upload(out_local, dst_gcs)
    print(f"[{name}] ✅ uploaded FINAL raster:", dst_gcs)

final_derived_paths = {}

for name, src in derived_clean.items():
    dst = f"{final_prefix}/{Path(src).name}"
    _rewrite_with_nodata_if_needed(name, src, dst, DEM_NODATA)
    final_derived_paths[name] = dst

print("\n✅ FINAL derived rasters written to:")
for k, v in final_derived_paths.items():
    print(" -", k, "->", v)

print("\nDONE ✅")




Local workdir: /tmp/sfincs_finalize_0snhd3kj

--- FINAL DEM (feet -> meters) ---
Reading CLEAN DEM: gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/clean/dep_dem_utm25m.tif
Writing FINAL DEM locally (GTiff): /tmp/sfincs_finalize_0snhd3kj/dep_dem_utm25m_meters.tif
✅ FINAL DEM local nodata: -9999.0
✅ FINAL DEM (meters) written to: gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/final/dep_dem_utm25m_meters.tif

--- FINAL DERIVED (repair nodata if missing) ---

[manning] Source: gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/clean/manning_n_utm25m.tif
[manning] Current nodata: None
[manning] Writing fixed raster locally: /tmp/sfincs_finalize_0snhd3kj/manning_n_utm25m.tif
[manning] ✅ fixed local nodata: -9999.0
[manning] ✅ uploaded FINAL raster: gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/final/manning_n_utm25m.tif

[impervious] Source: gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/clean/impervious_frac_utm25m.tif


In [9]:
# ============================================================
# Validate FINAL rasters (DEM meters) + CLEAN derived rasters
# Fixes Rasterio 404 by caching gs:// files locally via gcsfs
# ============================================================

import numpy as np
import rioxarray
import rasterio
import gcsfs
import tempfile
from pathlib import Path, PurePosixPath

EXPECTED_CRS = TARGET_CRS
EXPECTED_RES = float(TARGET_RES)

clean_prefix = f"{OUT_PREFIX}/rasters/clean"
final_prefix = f"{OUT_PREFIX}/rasters/final"

fs = gcsfs.GCSFileSystem()  # works on LEAP

# ------------------------------------------------------------
# Robustly pick FINAL DEM meters (as before)
# ------------------------------------------------------------
def gcs_ls(prefix: str):
    p = prefix.replace("gs://", "").rstrip("/")
    try:
        return fs.ls(p)
    except Exception:
        return []

def pick_final_dem():
    listing = gcs_ls(final_prefix)
    candidates = []
    for x in listing:
        name = PurePosixPath(x).name
        if name.endswith(".tif") and ("dep_dem" in name) and ("meter" in name or "meters" in name):
            candidates.append("gs://" + x)

    if len(candidates) == 1:
        return candidates[0]

    res_tag = f"utm{TARGET_RES}m"
    for c in candidates:
        if res_tag in c:
            return c

    return None

dem_final = pick_final_dem()
if dem_final is None:
    print("⚠️  Could not auto-find FINAL DEM meters in:", final_prefix)
    dem_final = f"{clean_prefix}/dep_dem_utm{TARGET_RES}m.tif"
    print("➡️  Falling back to CLEAN DEM:", dem_final)
else:
    print("✅ Using FINAL DEM meters:", dem_final)

paths_gcs = {
    "dep_dem": dem_final,
    "landcover":    f"{clean_prefix}/landcover_worldcover_utm{TARGET_RES}m.tif",
    "manning":      f"{final_prefix}/manning_n_utm{TARGET_RES}m.tif",
    "impervious":   f"{final_prefix}/impervious_frac_utm{TARGET_RES}m.tif",
    "curve_number": f"{final_prefix}/curve_number_cn_utm{TARGET_RES}m.tif",

}

# ------------------------------------------------------------
# Cache gs:// rasters to /tmp so rasterio doesn't 404
# ------------------------------------------------------------
cache_dir = Path(tempfile.mkdtemp(prefix="sfincs_validate_cache_"))
print("\nLocal cache dir:", cache_dir)

def cache_to_local(gs_path: str) -> Path:
    """
    Copy gs://bucket/path.tif -> /tmp/.../path.tif
    Returns local path to open with rasterio/rioxarray.
    """
    assert gs_path.startswith("gs://")
    rel = gs_path.replace("gs://", "")
    local_path = cache_dir / PurePosixPath(rel).name

    if not local_path.exists():
        # fs.get expects paths WITHOUT gs://
        fs.get(rel, str(local_path))

    return local_path

# Map keys -> local paths
paths_local = {}
print("\n--- Caching rasters locally ---")
for k, p in paths_gcs.items():
    if not fs.exists(p.replace("gs://", "")):
        raise FileNotFoundError(f"Missing file for {k}: {p}")
    lp = cache_to_local(p)
    paths_local[k] = lp
    print(f"{k:12s} -> {lp}  (from {p})")

# ------------------------------------------------------------
# Validation helpers (same logic as yours)
# ------------------------------------------------------------
def open_da_local(local_path: Path):
    return rioxarray.open_rasterio(str(local_path), masked=False).squeeze()

def stats_excluding_nodata(da):
    arr = np.asarray(da.values)
    nd = da.rio.nodata
    arr = arr[~np.isnan(arr)]
    if nd is not None:
        arr = arr[arr != nd]
    if arr.size == 0:
        return {"min": None, "max": None, "mean": None, "p50": None}
    return {
        "min": float(arr.min()),
        "max": float(arr.max()),
        "mean": float(arr.mean()),
        "p50": float(np.quantile(arr, 0.5)),
    }

def assert_close_tuple(a, b, tol=1e-6, label=""):
    for i, (x, y) in enumerate(zip(a, b)):
        if abs(float(x) - float(y)) > tol:
            raise AssertionError(f"{label} mismatch at index {i}: {a} vs {b}")

def validate(name, local_path: Path, ref=None, expect_dtype=None, expect_nodata=None):
    print(f"\n=== {name} ===")
    print("Local Path:", local_path)

    da = open_da_local(local_path).load()

    print("CRS:", da.rio.crs)
    print("Shape:", da.shape, "dtype:", da.dtype)
    print("Res:", da.rio.resolution())
    print("Bounds:", da.rio.bounds())
    print("NoData:", da.rio.nodata)

    # CRS + res checks
    if str(da.rio.crs) != str(EXPECTED_CRS):
        raise AssertionError(f"{name}: CRS mismatch")

    rx, ry = da.rio.resolution()
    if not (abs(abs(rx) - EXPECTED_RES) < 1e-6 and abs(abs(ry) - EXPECTED_RES) < 1e-6):
        raise AssertionError(f"{name}: resolution mismatch")

    # dtype
    if expect_dtype is not None and str(da.dtype) != str(expect_dtype):
        raise AssertionError(f"{name}: dtype mismatch (got {da.dtype}, expected {expect_dtype})")

    # nodata
    if expect_nodata is not None:
        if da.rio.nodata is None:
            raise AssertionError(f"{name}: nodata is None (expected {expect_nodata})")
        if not np.isclose(float(da.rio.nodata), float(expect_nodata)):
            raise AssertionError(f"{name}: nodata mismatch (got {da.rio.nodata}, expected {expect_nodata})")

    # NaNs should not exist
    if np.isnan(np.asarray(da.values)).any():
        raise AssertionError(f"{name}: contains NaNs")

    print("Stats:", stats_excluding_nodata(da))

    # alignment vs ref
    if ref is not None:
        if da.shape != ref.shape:
            raise AssertionError(f"{name}: shape mismatch vs ref")

        assert_close_tuple(
            tuple(da.rio.transform(recalc=True)),
            tuple(ref.rio.transform(recalc=True)),
            tol=1e-6,
            label=f"{name}: transform",
        )
        assert_close_tuple(
            da.rio.bounds(),
            ref.rio.bounds(),
            tol=1e-3,
            label=f"{name}: bounds",
        )
        print("✅ Alignment matches ref grid.")

    # Rasterio profile
    with rasterio.open(str(local_path)) as ds:
        prof = ds.profile
        print("Driver:", prof.get("driver"), "| Tiled:", prof.get("tiled"), "| Compress:", prof.get("compress"))
        print("Block:", prof.get("blockxsize"), prof.get("blockysize"), "| Overviews:", ds.overviews(1) or "None")

    return da

# ------------------------------------------------------------
# Run validations
# ------------------------------------------------------------
dep = validate("dep_dem_ref", paths_local["dep_dem"], ref=None, expect_dtype="float32", expect_nodata=-9999.0)

lc  = validate("landcover",    paths_local["landcover"],    ref=dep, expect_dtype="uint8",   expect_nodata=0)
mn  = validate("manning",      paths_local["manning"],      ref=dep, expect_dtype="float32", expect_nodata=-9999.0)
imp = validate("impervious",   paths_local["impervious"],   ref=dep, expect_dtype="float32", expect_nodata=-9999.0)
cn  = validate("curve_number", paths_local["curve_number"], ref=dep, expect_dtype="float32", expect_nodata=-9999.0)

print("\n✅ FINAL validation succeeded.")
print("Local cached files are in:", cache_dir)




✅ Using FINAL DEM meters: gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/final/dep_dem_utm25m_meters.tif

Local cache dir: /tmp/sfincs_validate_cache_5hxb5_vi

--- Caching rasters locally ---
dep_dem      -> /tmp/sfincs_validate_cache_5hxb5_vi/dep_dem_utm25m_meters.tif  (from gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/final/dep_dem_utm25m_meters.tif)
landcover    -> /tmp/sfincs_validate_cache_5hxb5_vi/landcover_worldcover_utm25m.tif  (from gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/clean/landcover_worldcover_utm25m.tif)
manning      -> /tmp/sfincs_validate_cache_5hxb5_vi/manning_n_utm25m.tif  (from gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/final/manning_n_utm25m.tif)
impervious   -> /tmp/sfincs_validate_cache_5hxb5_vi/impervious_frac_utm25m.tif  (from gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/final/impervious_frac_utm25m.tif)
curve_number -> /tmp/sfincs_validate_cache_5hxb5_vi/curve_number_cn_u